In [ ]:
import numpy as np
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML
from quick_sim import setup_sim
from edward_tools.coupled_fq_potential import coupled_flux_qubit_pot, coupled_flux_qubit_force, coupled_fq_pot
import edward_tools.fq_runner as fq_runner
from edward_tools.visualization import animate_sim_flux_qubit

import kyle_tools as kt
import matplotlib.pyplot as plt

import importlib, os
from edward_tools import coupled_fq_protocol_library, cfq_runner
from PARAMETER_INPUT import *
from PARAMETER_INPUT import _lambda, _theta, _eta

from edward_tools import coupled_fq_protocol_library
import edward_tools.cfq_batch_sweep as cfq_batch_sweep

coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
create_system_from_storage_and_computation_protocol = coupled_fq_protocol_library.create_system_from_storage_and_computation_protocol
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system


# 4. Szilard engine test

In [ ]:
import numpy as np
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython.display import HTML
from quick_sim import setup_sim
from edward_tools.coupled_fq_potential import coupled_flux_qubit_pot, coupled_flux_qubit_force, coupled_fq_pot
from edward_tools.visualization import animate_sim_flux_qubit
import kyle_tools as kt
import matplotlib.pyplot as plt
import importlib, os, hashlib, json
from edward_tools import coupled_fq_protocol_library, cfq_runner

In [ ]:
# from edward_tools import coupled_fq_protocol_library
# coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
# create_system = coupled_fq_protocol_library.create_system
# get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
# create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
# create_system_from_storage_and_computation_protocol = coupled_fq_protocol_library.create_system_from_storage_and_computation_protocol
# coupled_fq_runner = importlib.reload(cfq_runner)


In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params['N'] = 1_00
params['dt'] = 1.0/1_0


zeroDissipation = False
if zeroDissipation:
    _lambda = _lambda * 0
    _eta = _eta * 0


In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c0]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor

coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

coupled_fq_pot = Potential(coupled_flux_qubit_pot, coupled_flux_qubit_force, 14, 4,\
                           default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
# phi_1_dcx, phi_2_dcx = 3, 3
phi_1_dcx, phi_2_dcx = 0, 0
phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx

# gamma, beta_1, beta_2 = 0, 0, 0
gamma = 20
# d_beta_1, d_beta_2 = 0.6, 0.6
d_beta_1, d_beta_2 = 0, 0
params['sim_params'] = [_lambda, _theta, _eta]

# this is the initial parameter used
initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": 0,  "phi_2_x": 0,  "phi_1_dcx": phi_1_dcx,  "phi_2_dcx": phi_2_dcx,
        "M_12": 0, 'x_c': x_c
}


# Time symmetric protocol
# THis part is how you change the protocol. You can specify each substep by duration, the variables you want to 
# change and the name of the substep.
protocol_list = [
    {"duration":200,"phi_2_dcx":2.9,"name":"mix in y direction"},
    {"duration":100,"name":"fix"},
    {"duration":200,"M_12":-0.05,"name":"conditional tilt"},
    {"duration":100,"name":"fix"},
    {"duration":200,"phi_2_dcx":0,"name":"raise the barrier"},
    {"duration":100,"name":"fix"},
    {"duration":200,"M_12":0,"name":"4 well potential (constant)"},
    {"duration":100,"name":"fix"},
    {"duration":200,"phi_1_dcx":2.9,"name":"mix in x direction"},
    {"duration":100,"name":"fix"},
    {"duration":200,"phi_1_dcx":0,"M_12":0,"name":"4 well potential "},
    {"duration":100,"name":"4 well potential (constant)"}
]

In [ ]:

"""
# step 3: create the relevant storage protocol and computation protocol
"""
computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict)

In [ ]:
"""
# step 4: create the coupled_fq_runner
"""
cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol)
cfqr.initialize_sim()
cfqr.set_sim_attributes()
init_state_saved = cfqr.init_state

In [ ]:
manual_domain=[np.array([-5, -5])/time_scale_factor, np.array([5, 5])/time_scale_factor]
mapping_state_1_to_state_2_dict = {"00": ["00", "10"], "01": ["00", "10"], "10": ["01", "11"], "11": ["01", "11"]}

In [ ]:
# step 5: perform simulations

simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(params, initial_parameter_dict, protocol_list, \
                                        initial_state = init_state_saved, manual_domain = manual_domain, \
                                        phi_1_dcx = phi_1_dcx,  phi_2_dcx = phi_2_dcx)

cfqr = simResult["cfqr"]

In [ ]:
len(simResult["work_distribution"])

In [ ]:
# display work distribution

plt.hist(simResult["work_distribution"], bins = 45)
plt.show()
jarzyn_term = np.mean(np.exp(-simResult["work_distribution"]))
print(f'jarzyn = {jarzyn_term}')

In [ ]:
step_array = np.arange(simResult["work_statistic"][:,0].shape[0])
skip_step = int(len(step_array) * 0.05)
work_mean = simResult["work_statistic"][:,0]
work_std = simResult["work_statistic"][:,1]


plt.errorbar(step_array[::skip_step], work_mean[::skip_step], yerr = work_std[::skip_step])

# plt.errorbar(work_avg, work_error)

# M analysis

In [ ]:
M_12_value = protocol_list[2]["M_12"]
average_work = work_mean[-1]
fidelity_data = simResult["fidelity"]

saveData = {
    "M_12":         M_12_value,
    "fidelity":     fidelity_data,
    "average_work": average_work,
    "jarzyn_term":  jarzyn_term
}

json_path = "coupled_flux_qubit_protocol/M_analysis.json"
with open(json_path) as f:
    jsonData = json.load(f)
    jsonData.append(saveData)
    with open(json_path, "w+") as fw:
        json.dump(saveData, fw)

In [ ]:
simResult["fidelity"]

# dt_analysis

In [ ]:
import csv
sim_time = simResult["simulation_data"]["simulation_time"]

# with open('dt_data.csv', 'a+', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(["dt", "N", "jarzyn_term", "avg_work", "simulation_time"])
#     writer.writerow([params["dt"], params["N"], jarzyn_term, average_work, sim_time])

In [ ]:
# from IPython.display import HTML
from IPython import display
video = simResult['ani'].to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
# step 6: save the results

cfq_batch_sweep.saveSimulationResult(simResult, U0_1, timeOrStep = 'step', save = True, save_final_state = True)

In [ ]:
import csv
import pandas as pd

In [ ]:
df = pd.read_csv("dt_data.csv")

In [ ]:
N = 10000
df_N = df[df['N'] == N]
dt_data = df_N["dt"]
jarzyn_term_data = df_N["jarzyn_term"]
avg_work_data = df_N["avg_work"]
simulation_time_data = df_N["simulation_time"]

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,4))
fig.suptitle(f"N = {N}", fontsize=20, y=1.05)

ax[0].plot(dt_data, jarzyn_term_data, "x", markersize= 10)
# ax[0].set_ylabel("jarzyn_term", font)
ax[0].set_xscale("log")
ax[0].set_title('dt vs Jarzyn_term', fontsize=15)

ax[1].plot(dt_data, avg_work_data, "x", markersize= 10)
ax[1].set_xscale("log")
ax[1].set_title('dt vs avg work', fontsize=15)

ax[2].plot(dt_data, simulation_time_data, "x", markersize= 10)
ax[2].set_xscale("log")
ax[2].set_title('dt vs sim time (s)', fontsize=15)

plt.show()